<a href="https://colab.research.google.com/github/dasparagjyoti/Web-Data-Mining/blob/main/Web_Data_Mining_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Web Data Mining (CS684) | Programming Assignment |**

### 1. Write a program that takes an initial webpage url as an input and a number. It then finds all out links of the webpage and traverses the links in breadth first traversal till the shortest distance between a webpage and the initial webpage is within the given number. It outputs website url, heading and parent url in an excel file. There should be no repetition. A webpage should be occurring only once in the output. [10]

In [8]:
#Perform all necesary installation
!pip install fsspec

     |████████████████████████████████| 136 kB 5.2 MB/s 


In [9]:
# Import all necessary libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import re

In [11]:
# Function to take url as an input from the user
def input_url(message = ''):
  while True:
    url = input(message)
    if url.startswith('http://') or url.startswith('https://'):
      return url
    else:
      print('Invalid url. Please enter the url again!\n')

In [14]:
# Function to return heading of a webpage, given its url
def get_heading(url):
  try:
    html = urlopen(Request(url, headers={'User-Agent':'Mozilla'}))
    bs = BeautifulSoup(html, 'html.parser')
    heading = bs.find('title').get_text()
  except:
    heading = ''
  return heading

In [15]:
# Function to return web crawl information of a single level
def crawl_single_level(curr_url, curr_level):
  m = re.search('https?:\/\/[\w]+\.([A-Za-z_0-9.-]+).*', curr_url)
  parent_domain =  m.group(1) if m else ''
  total_links_found = 0
  links = []
  headings = []
  
  try:
    html = urlopen(curr_url)
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a'):
      if 'href' in link.attrs and link.attrs['href'].startswith('http'):
        if re.search('https?:\/\/[\w]+\.([A-Za-z_0-9.-]+).*', link.attrs['href']).group(1) != parent_domain:
          links.append(link.attrs['href'])
          heading = get_heading(link.attrs['href'])
          headings.append(heading)
  except:
    pass

  total_links_found = len(links)
  parent_url = curr_url

  return curr_level+1, parent_url, total_links_found, links, headings

In [16]:
# Crawl webpages via breadth first traversal
def web_crawl(initial_url, level):
  # Current index of link being crawled
  current_index = 0

  # Lists to store the crawled information
  level_list = []
  url_list = []
  parent_url_list = []
  heading_list = []

  # Initialize root node
  level_list.append(0)
  url_list.append(initial_url)
  parent_url_list.append('')
  heading_list.append(get_heading(initial_url))

  # Start crawling level by level
  if level > 0:
    while current_index < len(level_list):
      if level_list[current_index] >= level:
        break

      curr_level, parent_url, total_links_found, links, headings = crawl_single_level(curr_url = url_list[current_index], curr_level = level_list[current_index])
      
      for i in range(total_links_found):
        level_list.append(curr_level)
        url_list.append(links[i])
        parent_url_list.append(parent_url)
        heading_list.append(headings[i])

      current_index += 1

  # Create Pandas dataframe out of the lists
  dictn = {'level': level_list, 'url': url_list, 'parent_url': parent_url_list, 'heading': heading_list}
  df = pd.DataFrame(dictn)

  # Remove duplicate url containing rows
  df.drop_duplicates(subset=['url'], inplace=True)

  # Reset index of the DataFrame
  df = df.reset_index(drop=True)

  return df

In [17]:
# Take user input and call the web_crawl function
url = input_url('Enter a url (E.g. https://www.geeksforgeeks.org/): ')
crawl_level = int(input('Choose a number (above 0): '))
df = web_crawl(initial_url = url, level = crawl_level)

Enter a url (E.g. https://www.geeksforgeeks.org/): https://www.w3schools.com/
Choose a number (above 0): 30


In [18]:
df

,level,url,parent_url,heading
0,0,https://www.w3schools.com/,,W3Schools Online Web Tutorials
1,1,https://discord.gg/6Z7UaRbUQM,https://www.w3schools.com/,w3schools
2,1,https://www.linkedin.com/company/w3schools.com/,https://www.w3schools.com/,
3,1,https://www.instagram.com/w3schools.com_official/,https://www.w3schools.com/,\nLogin • Instagram\n
4,1,https://www.facebook.com/w3schoolscom/,https://www.w3schools.com/,w3schools.com - 首页


In [14]:
# Convert DataFrame into excel file
df.to_excel('assignment-1-results.xlsx', index = False)

###2. Write a program that takes input a phrase, a webpage and a number, n; and then outputs all the webpages that match the phrase (exact phrase match), are within ‘n’ distance from the input webpage. The order in which output is displayed should be according to cosine similarity between the initial phrase and the webpage content. The output is should be stored in excel format by the program. [10]

In [1]:
# Import all necessary libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as numpy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [2]:
nltk.download(['punkt', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Function to get cosine similarity between a given phrase
# and the contents of a webpage
def get_cosine_similarity(phrase, url):
  X = phrase.lower()
  Y = ''
  try:
    html = urlopen(Request(url, headers={'User-Agent':'Mozilla'}))
    bs = BeautifulSoup(html, 'html.parser')

    # Remove script and style tags
    for data in bs(['style', 'script']):
      data.decompose()

    text = ' '.join(bs.stripped_strings).lower()
    Y = text
  except:
    pass
  
  # Tokenization
  X_tok = word_tokenize(X)
  Y_tok = word_tokenize(Y)

  # Clean the list of tokens
  Y_tok  = [re.sub(r'[^a-zA-Z0-9]','',string) for string in Y_tok]
  Y_tok = ' '.join(Y_tok).split()
             
  # print('X tokenized list = ', X_tok)
  # print('Y tokenized list = ', Y_tok)

  # Remove stopwords from the sentences
  sw = stopwords.words('english')
  l1 = []
  l2 = []
  X_set = {word for word in X_tok if not word in sw} 
  Y_set = {word for word in Y_tok if not word in sw}

  # Form a set containing keywords of both strings
  rvector = X_set.union(Y_set)

  for word in rvector:
    if word in X_set:
      l1.append(1)
    else:
      l1.append(0)

    if word in Y_set:
      l2.append(1)
    else:
      l2.append(0)

  dot_product = 0

  # Cosine similarity formula
  for i in range(len(rvector)):
    dot_product += l1[i]*l2[i]
  
  cosine_similarity = dot_product / float((sum(l1)*sum(l2))**0.5)

  return cosine_similarity

In [4]:
def phrase_match(phrase, url):
  matched = False
  try:
    html = urlopen(Request(url, headers={'User-Agent':'Mozilla'}))
    bs = BeautifulSoup(html, 'html.parser')
    # Remove script and style tags
    for data in bs(['style', 'script']):
      data.decompose()

    text = ''.join(bs.stripped_strings) 

    if text.find(phrase) >= 0:
      matched = True
  except:
    pass

  return matched

In [5]:
def create_cosine_similarity_df(initial_url, level):
  message = 'Cosine similarity df created successfully!'
  # Perform web crawling till the given number of 'n' levels is reached
  df1 = web_crawl(initial_url = initial_url, level = level)

  # Calculate phrase match value for all webpages in the given 'n' levels
  df1['phrase_match'] = df1.apply(lambda row: phrase_match(phrase = phrase, url = row['url']), axis = 1)
  df1 = df1[df1['phrase_match'] == True]

  # Modify the dataframe to drop unnecessary columns and
  # Insert a column for cosine similarity
  df1 = df1[['level', 'url']]
  df1['cosine_similarity'] = 0

  # If zero webpages match with the given phrase
  if len(df1) == 0:
    return df1, 'Zero webpages matched with the given phrase.'
  
  # Calculate cosine similarity for the phrase matched webpages
  df1['cosine_similarity'] = df1.apply(lambda row: get_cosine_similarity(phrase = phrase, url = row['url']), axis = 1)

  # Sort the dataframe by cosine similarity value
  df1.sort_values(by = ['cosine_similarity'], inplace = True, ascending = False)

  # Reset index of the sorted DataFrame
  df1 = df1.reset_index(drop=True)

  return df1, message

In [24]:
url_ = input_url('Enter a url (E.g. https://www.geeksforgeeks.org/): ')
phrase = input('Enter phrase: ')
n = int(input ('Choose a number (above 0): '))

Enter a url (E.g. https://www.geeksforgeeks.org/): https://www.w3schools.com/
Enter phrase: Programming Languages
Choose a number (above 0): 4


In [25]:
cos_sim_df, message = create_cosine_similarity_df(initial_url = url_, level = n)

In [22]:
print(message)

Cosine similarity df created successfully!


In [26]:
cos_sim_df

,level,url,cosine_similarity


In [ ]:
# Convert DataFrame into excel file
cos_sim_df.to_excel('assignment-2-results.xlsx', index = False)